In [1]:
# Importing required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from glob import glob
import re
import openpyxl

In [ ]:
# Extracting text from articles and storing in a dictionary with keys as file_URL_id

articles = pd.read_excel("Input.xlsx")
files = {}

for index,url_ in enumerate(articles['URL']):
    response = requests.get(url_)
    files[f"file{index+37}"] = ""
    if response.status_code != 404:
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.find("h1")
        print(index+37, title.text)
        files[f"file{index+37}"] = title.get_text()+ ' '
        div_elements = soup.find_all("div", class_="td-post-content tagdiv-type")
        for div_element in div_elements:
            if div_element.p != None:
                if div_element.pre != None:
                    div_element.pre.extract()
                files[f"file{index+37}"] += div_element.get_text()   # Get the text content of each div element
                class1+=1
        else:
            div_elements = soup.find_all("div", class_="tdb-block-inner td-fix-index")
            for div_element in div_elements:
                if div_element.p != None:
                    if div_element.pre != None:
                        div_element.pre.extract()
                    files[f"file{index+37}"] += div_element.get_text() # Get the text content of each div element
                    class2+=1
    else:
        print(index+37,"Error: The request failed.")
        error+=1


In [ ]:
# Finding output variables

def count_syllables(word):
    # Count the number of vowels in the word.
    num_vowels = len(re.findall(r'[aeiou]', word.lower()))

    # Handle exceptions like words ending with "es" or "ed".
    if word.endswith('es') or word.endswith('ed'):
        num_vowels -= 1
    # Return the number of syllables in the word.
    return num_vowels

def count_personal_pronouns(text):
    # Create a regular expression to match personal pronouns.
    pattern = re.compile(r'\b(I|we|my|ours|our|your|yours|(?-i:us))\b', re.I)

    # Count the number of matches.
    count = len(re.findall(pattern, text))

    # Return the count.
    return count


# STOP WORDS LIST
stop_words = ""
stop_words_path_pattern = "StopWords\\StopWords_*.txt"
stop_words_files = glob(stop_words_path_pattern)
for stop_words_file in stop_words_files:
    with open(stop_words_file, 'r') as file:
        # Read the contents of the file
        stop_words += file.read()
stop_words_list = set(stop_words.lower().split())
sorted_stop_words_list = sorted(stop_words_list)

# POSITIVE WORDS LIST
pos_words_path = "MasterDictionary\\positive-words.txt"
with open(pos_words_path,"r") as pos:
    sorted_positive_words = pos.read().lower().split()

# NEGATIVE WORDS LIST
neg_words_path = "MasterDictionary\\negative-words.txt"
with open(neg_words_path,"r") as neg:
    sorted_negative_words = neg.read().lower().split()

# SCORES
positive_score = 0
negative_score = 0
polarity_score = 0
remain = 0
total_words = 0 
total_sent = 0
complex_words = []
avg_word_length = 0
personal_pronouns = 0
total_syllables_count = 0

output = {"POSITIVE SCORE":[],"NEGATIVE SCORE":[],"POLARITY SCORE":[],"SUBJECTIVITY SCORE":[],"AVG SENTENCE LENGTH":[],\
          "PERCENTAGE OF COMPLEX WORDS":[],"FOG INDEX":[],"AVG NUMBER OF WORDS PER SENTENCE":[],"COMPLEX WORD COUNT":[]\
          ,"WORD COUNT":[],"SYLLABLE PER WORD":[],"PERSONAL PRONOUNS":[],"AVG WORD LENGTH":[]}

In [ ]:
for file in files:
    final_text = []
    text = files[file]
    personal_pronouns = count_personal_pronouns(text)
    text_words_punctn = nltk.word_tokenize(text.lower())
    text_words = [word for word in text_words_punctn if not re.match(r'[.,|?!0-9]', word)]
    text_length = sum(len(word) for word in text_words)
    total_words = len(text_words)
    text_sent = nltk.sent_tokenize(text)
    total_sent = len(text_sent)
    if text_words != None:
        for word in text_words:
            total_syllables_count+=count_syllables(word)
            if (count_syllables(word)) > 2:
                complex_words.append(word)
            if word not in sorted_stop_words_list:
                final_text.append(word)
                if word in sorted_positive_words:
                    positive_score+=1
                elif word in sorted_negative_words:
                    negative_score+=1
                else:
                    remain+=1
            words_after_cleaning = len(final_text)
            polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
            subjectivity_score = (positive_score + negative_score)/ ((words_after_cleaning) + 0.000001)
            avg_sent_length = total_words/total_sent
            complex_words_count = len(complex_words)
            avg_word_length = text_length/total_words
            percent_of_complex_words = complex_words_count/total_words
            fog_index = 0.4*(avg_sent_length+percent_of_complex_words)
            syllable_per_word = total_syllables_count/total_words
        output["POSITIVE SCORE"].append(positive_score)
        output["NEGATIVE SCORE"].append(negative_score)
        output["POLARITY SCORE"].append(polarity_score)
        output["SUBJECTIVITY SCORE"].append(subjectivity_score)
        output["AVG SENTENCE LENGTH"].append(avg_sent_length)
        output["PERCENTAGE OF COMPLEX WORDS"].append(percent_of_complex_words)
        output["FOG INDEX"].append(fog_index)
        output["AVG NUMBER OF WORDS PER SENTENCE"].append(avg_sent_length)
        output["COMPLEX WORD COUNT"].append(complex_words_count)
        output["WORD COUNT"].append(words_after_cleaning)
        output["SYLLABLE PER WORD"].append(syllable_per_word)
        output["PERSONAL PRONOUNS"].append(personal_pronouns)
        output["AVG WORD LENGTH"].append(avg_word_length)
        print(file,"complete")


In [ ]:
# Exporting the data in output dictionary to excel file

wb = openpyxl.load_workbook("Output Data Structure.xlsx")

sheet = wb.active

for col in range(3, 16):
    for row in range(2, 110):
        char = chr(64 + col)
        keys = list(output.keys())
        col_index = keys[col-3]
        
        sheet[char+str(row)] = round(output[col_index][row-2],2)
wb.save("Output Data Structure.xlsx")